# DEMO#1 - 01/31/2024 - See https://jh-ml.nccs.nasa.gov/jupyterhub-prism/user/gtamkin/lab/workspaces/auto-0/tree/_AGB-dev/mpf-model-factories/MultiPathFusion/multi_path_fusion/notebooks/glenn/gt_mc_get_stats.ipynb

# DEMO#2 - Expand Experiments...

# Generate Permutation Importance (i.e., rank) from SHAP files

In [1]:
import shap
import numpy as np
import pickle
import glob
import pandas as pd
import subprocess
import csv
from pathlib import Path

In [2]:
def format_band_set(subset):
    indexListIntStr = ''
    count = 0
    for band in subset:
        if (count == 0):
            indexListIntStr = indexListIntStr + str(band)
        else:
            indexListIntStr = indexListIntStr + ', ' + str(band)
        count = count + 1
    return indexListIntStr


# Read from CSV

In [3]:
hyperspectral_aggregate_bandnames = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate-bandnames.csv'
only_row = None
with open(hyperspectral_aggregate_bandnames) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        only_row = row
        print(row[0])
        print(row[446])
#print(only_row)

ARI
2511.7


In [4]:
# initialize list of lists
headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
site = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0'

## Read batch of shap files for experiment, which contains run-time artifacts (e.g., SHAP values)

In [16]:
root_data_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output'

In [41]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_ALL = [
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','randit','447', '10897', '10', '200','141:13:25','ilab213','40','Pool', 
      'randint_sets_20240217_collection-ALL_min_occurrences-200_batch_size-10_num_samples-10897_bandLen-447','866122034458507573', 
      'ALL on ilab213 (in serial) - min 200 occurrences: ~96 hours',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','randit','447', '5737', '10', '100','96:13:25','ilab213','40','Pool', 
      'randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447','218557492334736085', 
      'ALL on ilab213 - min 100 occurrences: ~96 hours',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '3227', '10', '50','96:13:25','ilab213','40','Pool', 
      'random_sets_20240209_10trials_collection-ALL_min_occurrences-50_batch_size-10_num_samples-3227_bandLen-447','961551132591233530', 
      'ALL on ilab201-212 - min 50 occurrences: ~48 hours',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','randit','447', '922', '10', '10','96:13:25','gpu004','40','Pool', 
      'randint_sets_20240220_collection-ALL_max_occurrences-10_batch_size-10_num_samples-922_bandLen-447','814952172296500040', 
      'ALL on gpu004 - min 10 occurrences: ~96 hours ',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ','EarlyWarning'],
]

In [42]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_NEON = [
     ['NEON','NoTop10Index', 'Top10Rank', 'DivBy10K','randit','426', '5666', '10', '100', '30:51:00.00','gpu004','40','Pool', 
      'randint_sets_20240215_collection-NEON_max_occurrences-100_batch_size-10_num_samples-5666_bandLen-426','188577871478003488', 
      'NEON on gpu004 - min 100 occurrences: ~31 hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
     ['NEON','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','426', '3153', '10', '50', '52:49:00.00','ilab202-212','10','Pool', 
      'random_sets_20240205_collection-NEON_max_occurrences-50_batch_size-10_num_samples-3153_bandLen-426','980360802055833747', 
      'NEON on ilab213 - min 50 occurrences: ~53 hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
]


In [8]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_VI = [
     ['VI','NoTop10Index', 'Top10Rank', 'DivBy10K','randit','21', '261', '10', '100','26:00:00.00','ilab213','40','Pool', 
      'ilab213-finale-VI','681206886322137043', 'VI on ilab213 - min 100 occurrences: ~26 hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','21', '128', '10', '10','?','ilab213','40','Pool', 
      'random_sets_ilab213_20240205_20trials_collection-VI_max_occurrences-10_batch_size-10_num_samples-128_bandLen-447','855815048022458276', 
      'FINISHED VIs:* Determine how long it will take @ 20 tasks [10 indices per subset]/1 node [30 CPUs] /60 minutes',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','21', '128', '10', '10','?','ilabs202-212','10','Pool', 
      'random_sets_20240204_20trials_collection-VI_max_occurrences-10_batch_size-10_num_samples-128_bandLen-447','390822361515712409', 
      '# FINISHED: VI on ilabs202-212 - ~3 hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
]


In [9]:
# initialize list of lists
root_data_path = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate'
site = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0'

#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_v1 = [
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', '10:13:32.980461','gpu001','40','Pool', '202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax','225919640119481430', 
      '### gpu004: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1089', '10', '10', '9:44:57.880395','gpu001','40','Pool', '202410125-gpu001-callbacks-divideby10k_noMinMax','393911649901623133', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', '9:57:19.194150','gpu001','40','Pool', '202410121b-gpu001-callbacks-divideby10k_noMinMax','971589025605583160', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', '9:38:07.808645','gpu001','40','Pool', '202410121-gpu001-callbacks-divideby10k_noMinMax','916501809805875784', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', '9:29:50.877797','gpu001','40','Pool', '202410120-gpu001-callbacks-divideby10k_noMinMax','871912588490665214', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', 'NoMinutes','ilab[201-212,211]','10','pdsh', '202410120-callbacks-divideby10k_noMinMax','648862663793551468', 
      '### CPU pdsh[201-212,211]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'DivBy10K','sample','447', '1088', '10', '10', '4:24:16.993903','ilab[201-212]','10','pdsh', '20241019-callbacks-divideby10k_noMinMax','964586160942144948', 
      '### CPU pdsh[201-212]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank','minmax','sample','447', '1088', '10', '10', '9:36:55.985009','gpu003','40','Pool', '20240118-callbacks-gpu','504373152637067057', 
      'gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank','minmax','sample','447', '1088', '10', '10', '9:34:11.607509','gpu003','40','Pool', '20240117-callbacks-gpu','660588493840123655', 
      'gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank', 'minmax','sample','447', '1088', '10', '10', '9:34:11.607509','gpu003','40','Pool', '20240113_gpu','342218863809278249', 
      'GPU: Epochs = 3 and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
     ['ALL','NoTop10Index', 'Top10Rank','minmax','sample','447', '1088', '10', '10', '9:34:11.607509','gpu003','40','Pool', '20240112','789617681324607598', 
      'CPU & GPU: Epochs = 2 and explainer.shap_values(X.iloc[0:50, :], nsamples=500) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
]

In [43]:
#permutation_results = permutation_results_ALL
#permutation_results = permutation_results_VI
#permutation_results = permutation_results_NEON
permutation_results = permutation_results_ALL + permutation_results_v1 + permutation_results_NEON + permutation_results_VI

print(len(permutation_results), '\n',permutation_results)

20 
 [['ALL', 'NoTop10Index', 'Top10Rank', 'DivBy10K', 'randit', '447', '10897', '10', '200', '141:13:25', 'ilab213', '40', 'Pool', 'randint_sets_20240217_collection-ALL_min_occurrences-200_batch_size-10_num_samples-10897_bandLen-447', '866122034458507573', 'ALL on ilab213 (in serial) - min 200 occurrences: ~96 hours', 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ', 'EarlyWarning'], ['ALL', 'NoTop10Index', 'Top10Rank', 'DivBy10K', 'randit', '447', '5737', '10', '100', '96:13:25', 'ilab213', '40', 'Pool', 'randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447', '218557492334736085', 'ALL on ilab213 - min 100 occurrences: ~96 hours', 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif ', 'EarlyWarning'], ['ALL', 'NoTop10Index', 'Top10Rank', 'DivBy10K', 'sample', '447', '3227', '10', '50', '96:13:25', 'ilab213', '40', 'Pool', 'random_

In [44]:
# Create the pandas DataFrame
df = pd.DataFrame(permutation_results, columns=headers)
df

,Group,Top10Index,Top10Rank,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,ALL,NoTop10Index,Top10Rank,DivBy10K,randit,447,10897,10,200,141:13:25,ilab213,40,Pool,randint_sets_20240217_collection-ALL_min_occurrences-200_batch_size-10_num_samples-10897_bandLen-447,866122034458507573,ALL on ilab213 (in serial) - min 200 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
1,ALL,NoTop10Index,Top10Rank,DivBy10K,randit,447,5737,10,100,96:13:25,ilab213,40,Pool,randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447,218557492334736085,ALL on ilab213 - min 100 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
2,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,3227,10,50,96:13:25,ilab213,40,Pool,random_sets_20240209_10trials_collection-ALL_min_occurrences-50_batch_size-10_num_samples-3227_bandLen-447,961551132591233530,ALL on ilab201-212 - min 50 occurrences: ~48 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
3,ALL,NoTop10Index,Top10Rank,DivBy10K,randit,447,922,10,10,96:13:25,gpu004,40,Pool,randint_sets_20240220_collection-ALL_max_occurrences-10_batch_size-10_num_samples-922_bandLen-447,814952172296500040,ALL on gpu004 - min 10 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
4,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1088,10,10,10:13:32.980461,gpu001,40,Pool,202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax,225919640119481430,"### gpu004: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
5,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1089,10,10,9:44:57.880395,gpu001,40,Pool,202410125-gpu001-callbacks-divideby10k_noMinMax,393911649901623133,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
6,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1088,10,10,9:57:19.194150,gpu001,40,Pool,202410121b-gpu001-callbacks-divideby10k_noMinMax,971589025605583160,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
7,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1088,10,10,9:38:07.808645,gpu001,40,Pool,202410121-gpu001-callbacks-divideby10k_noMinMax,916501809805875784,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
8,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1088,10,10,9:29:50.877797,gpu001,40,Pool,202410120-gpu001-callbacks-divideby10k_noMinMax,871912588490665214,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
9,ALL,NoTop10Index,Top10Rank,DivBy10K,sample,447,1088,10,10,NoMinutes,"ilab[201-212,211]",10,pdsh,202410120-callbacks-divideby10k_noMinMax,648862663793551468,"### CPU pdsh[201-212,211]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning


# Loop through result files and create the ranking

In [46]:
#. Loop through result files and create the ranking
for r_index, row in df.iterrows():
    rank_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/' + df.at[r_index,'ResultDir'] + '.rank' 
    my_file = Path(rank_file)
    if my_file.is_file():
        bandAbsSumArr = pickle.load(open(rank_file, "rb"))
        print('\n ======== index[', r_index, '] ==========')
        print('Loaded:', rank_file)
    else:
        shap_dir_prefix = root_data_path + '/' + df.at[r_index,'ResultDir'] + '/' + df.at[r_index,'ID'] 
        shap_dir = shap_dir_prefix + '/PERMUTATION_IMPORTANCE_VALUES/'
        print('\n ======== index[', r_index, '] ==========')
        print(r_index,'index: ', shap_dir)

        # Returns a list of names in list files. 
        shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                           recursive = True) 

        # set to max number of hyperspectral indices in experiment
        bandLen = int(df.at[r_index,'#Bands'])
        bandOccurenceArr = np.zeros(bandLen).astype(int)
        bandAbsSumArr = np.zeros(bandLen)

        first_time = True
        index = 0
        # Reload config
        start = left = '[['
        end = right = ']]'

        for shap_vals_path in shap_files:

            index = index+1
            num_shap_files = len(shap_files)
            bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
            bandList = [e for e in bandListStr.split(',')]
            for bandNum in bandList:
                bandOccurenceArr[int(bandNum)-1] = (bandOccurenceArr[int(bandNum)-1]) + 1

            # Reload shap values
            shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
            num_bins = len(shap_values0to50)
            num_rows = len(shap_values0to50[0])
            num_bands = num_cols = len(shap_values0to50[0][0])

            #  Walk across bins [0,4]
            for _bin in range(0, num_bins):

                # get absolute value of all values in matrix
                shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])

                # convert numpy matrix to dataframe and print sum of absolute values in each column
                df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)

                # create a row (numpy array) where each column is the sum of the absolute values in that vertical column
                shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)

                # sort the row from low to high to determine the rank of the columns (e.g., features)
                shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)

                # loop thru each column and update the corresponding statistics in the global row
                for _col in range(0, num_cols):
                    bandNum = int(bandList[_col])

                    # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
                    # this allows us to keep a running total shap values per feature for downselection later....
                    bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + df_shap_values0to50_bin_abs[_col].sum())

        ascending_indices = np.argsort(bandAbsSumArr)[:10]
        minIndex = ascending_indices[0]

        descending_indices = np.argsort(bandAbsSumArr)[::-1][:10]
        maxIndex = descending_indices[0]

        pickle.dump(bandAbsSumArr, open(rank_file , "wb"))
        print('Saved:', rank_file)
        
    with np.printoptions(precision=5, suppress=True):
        print('Final:', bandAbsSumArr[:10])
        print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
        print('Final L->H:', np.argsort(bandAbsSumArr)[:10])
        row['Top10Index'] = np.argsort(bandAbsSumArr)[::-1][:10]

        indexList = np.argsort(bandAbsSumArr)[::-1][:10]
        bandList = []
        for index in range(0, 10):
        #for (index < 10):
            bandList.append(str(only_row[indexList[index]]))
            print(indexList[index], only_row[indexList[index]])        
            index = index + 1
        row['Top10Rank'] = bandList

        print('Final H->L:', row['Top10Index'] )
        print('Final max:', row['Top10Index'][0])


 ======== index[ 0 ] ==========
Loaded: /explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/randint_sets_20240217_collection-ALL_min_occurrences-200_batch_size-10_num_samples-10897_bandLen-447.rank
Final: [ 0.06858  0.00333  0.07321  0.15559  1.95573  2.05512 23.55394 20.11544
 16.74901  0.7377 ]
Final.min: 0.003333725624965147 Final.max: 175.03915941981288
Final L->H: [ 1 29 28 25 30 24 23 31 26 38]
15 NIRv
6 fPAR
7 LAI
8 MCTI
326 1910.6757
324 1900.6586
320 1880.6244
325 1905.6671
219 1374.7622
321 1885.633
Final H->L: [ 15   6   7   8 326 324 320 325 219 321]
Final max: 15

 ======== index[ 1 ] ==========
Loaded: /explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447.rank
Final: [ 0.03721  0.00188  0.05182  0.09636  1.28383  0.91212 22.80987 24.41553
 11.58753  0.52776]
Final.min: 0.0018750489476317658 Final.max: 141.05498961206217
Final L->H: [ 1 29 26 24 25 3

In [47]:
dfTest = df
dfTest

,Group,Top10Index,Top10Rank,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,ALL,"[15, 6, 7, 8, 326, 324, 320, 325, 219, 321]","[NIRv, fPAR, LAI, MCTI, 1910.6757, 1900.6586, 1880.6244, 1905.6671, 1374.7622, 1885.633]",DivBy10K,randit,447,10897,10,200,141:13:25,ilab213,40,Pool,randint_sets_20240217_collection-ALL_min_occurrences-200_batch_size-10_num_samples-10897_bandLen-447,866122034458507573,ALL on ilab213 (in serial) - min 200 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
1,ALL,"[15, 7, 6, 8, 326, 320, 219, 324, 323, 321]","[NIRv, LAI, fPAR, MCTI, 1910.6757, 1880.6244, 1374.7622, 1900.6586, 1895.65, 1885.633]",DivBy10K,randit,447,5737,10,100,96:13:25,ilab213,40,Pool,randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447,218557492334736085,ALL on ilab213 - min 100 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
2,ALL,"[15, 7, 8, 326, 320, 219, 220, 321, 325, 324]","[NIRv, LAI, MCTI, 1910.6757, 1880.6244, 1374.7622, 1379.7708, 1885.633, 1905.6671, 1900.6586]",DivBy10K,sample,447,3227,10,50,96:13:25,ilab213,40,Pool,random_sets_20240209_10trials_collection-ALL_min_occurrences-50_batch_size-10_num_samples-3227_bandLen-447,961551132591233530,ALL on ilab201-212 - min 50 occurrences: ~48 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
3,ALL,"[15, 7, 6, 8, 217, 216, 21, 220, 219, 321]","[NIRv, LAI, fPAR, MCTI, 1364.7452, 1359.7366, 383.072, 1379.7708, 1374.7622, 1885.633]",DivBy10K,randit,447,922,10,10,96:13:25,gpu004,40,Pool,randint_sets_20240220_collection-ALL_max_occurrences-10_batch_size-10_num_samples-922_bandLen-447,814952172296500040,ALL on gpu004 - min 10 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
4,ALL,"[15, 7, 6, 8, 221, 217, 216, 218, 223, 323]","[NIRv, LAI, fPAR, MCTI, 1384.7793, 1364.7452, 1359.7366, 1369.7537, 1394.7964, 1895.65]",DivBy10K,sample,447,1088,10,10,10:13:32.980461,gpu001,40,Pool,202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax,225919640119481430,"### gpu004: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
5,ALL,"[15, 6, 7, 8, 218, 217, 216, 221, 223, 323]","[NIRv, fPAR, LAI, MCTI, 1369.7537, 1364.7452, 1359.7366, 1384.7793, 1394.7964, 1895.65]",DivBy10K,sample,447,1089,10,10,9:44:57.880395,gpu001,40,Pool,202410125-gpu001-callbacks-divideby10k_noMinMax,393911649901623133,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
6,ALL,"[15, 7, 8, 218, 216, 221, 217, 223, 323, 219]","[NIRv, LAI, MCTI, 1369.7537, 1359.7366, 1384.7793, 1364.7452, 1394.7964, 1895.65, 1374.7622]",DivBy10K,sample,447,1088,10,10,9:57:19.194150,gpu001,40,Pool,202410121b-gpu001-callbacks-divideby10k_noMinMax,971589025605583160,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
7,ALL,"[15, 7, 6, 8, 218, 216, 221, 217, 223, 323]","[NIRv, LAI, fPAR, MCTI, 1369.7537, 1359.7366, 1384.7793, 1364.7452, 1394.7964, 1895.65]",DivBy10K,sample,447,1088,10,10,9:38:07.808645,gpu001,40,Pool,202410121-gpu001-callbacks-divideby10k_noMinMax,916501809805875784,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-ba

#df

In [48]:
print('[ID]-[Group]-[Min]-[Band]-[Trials]-[Scaling]-[Top 10 Rank]-[Top 10 Indices]-[Randomizer]')
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
for index, row in df.iterrows():
    print('', str(index).rjust(2), '-',row['Group'].rjust(4),'-', row['MinBands'].rjust(3), '-', row['#Bands'].rjust(3),'-', row['#Trials'].rjust(4), '-', row['Scaling'].rjust(8), '-', row['Top10Rank'], '-', row['Top10Index'], '-', row['Randomizer'])

[ID]-[Group]-[Min]-[Band]-[Trials]-[Scaling]-[Top 10 Rank]-[Top 10 Indices]-[Randomizer]
  0 -  ALL - 200 - 447 - 10897 - DivBy10K - ['NIRv', 'fPAR', 'LAI', 'MCTI', '1910.6757', '1900.6586', '1880.6244', '1905.6671', '1374.7622', '1885.633'] - [ 15   6   7   8 326 324 320 325 219 321] - randit
  1 -  ALL - 100 - 447 - 5737 - DivBy10K - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1910.6757', '1880.6244', '1374.7622', '1900.6586', '1895.65', '1885.633'] - [ 15   7   6   8 326 320 219 324 323 321] - randit
  2 -  ALL -  50 - 447 - 3227 - DivBy10K - ['NIRv', 'LAI', 'MCTI', '1910.6757', '1880.6244', '1374.7622', '1379.7708', '1885.633', '1905.6671', '1900.6586'] - [ 15   7   8 326 320 219 220 321 325 324] - sample
  3 -  ALL -  10 - 447 -  922 - DivBy10K - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1364.7452', '1359.7366', '383.072', '1379.7708', '1374.7622', '1885.633'] - [ 15   7   6   8 217 216  21 220 219 321] - randit
  4 -  ALL -  10 - 447 - 1088 - DivBy10K - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1384.7793', '

In [49]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.colheader_justify', 'left')
dfSummary = df[['Group','MinBands','#Bands','#Trials','Randomizer','Scaling','Top10Rank']] 
dfSummary.style.set_properties(**{'text-align': 'left'})
dfSummary

,Group,MinBands,#Bands,#Trials,Randomizer,Scaling,Top10Rank
0,ALL,200,447,10897,randit,DivBy10K,"[NIRv, fPAR, LAI, MCTI, 1910.6757, 1900.6586, 1880.6244, 1905.6671, 1374.7622, 1885.633]"
1,ALL,100,447,5737,randit,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1910.6757, 1880.6244, 1374.7622, 1900.6586, 1895.65, 1885.633]"
2,ALL,50,447,3227,sample,DivBy10K,"[NIRv, LAI, MCTI, 1910.6757, 1880.6244, 1374.7622, 1379.7708, 1885.633, 1905.6671, 1900.6586]"
3,ALL,10,447,922,randit,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1364.7452, 1359.7366, 383.072, 1379.7708, 1374.7622, 1885.633]"
4,ALL,10,447,1088,sample,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1384.7793, 1364.7452, 1359.7366, 1369.7537, 1394.7964, 1895.65]"
5,ALL,10,447,1089,sample,DivBy10K,"[NIRv, fPAR, LAI, MCTI, 1369.7537, 1364.7452, 1359.7366, 1384.7793, 1394.7964, 1895.65]"
6,ALL,10,447,1088,sample,DivBy10K,"[NIRv, LAI, MCTI, 1369.7537, 1359.7366, 1384.7793, 1364.7452, 1394.7964, 1895.65, 1374.7622]"
7,ALL,10,447,1088,sample,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1369.7537, 1359.7366, 1384.7793, 1364.7452, 1394.7964, 1895.65]"
8,ALL,10,447,1088,sample,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1359.7366, 1364.7452, 1369.7537, 1384.7793, 1394.7964, 1910.6757]"
9,ALL,10,447,1088,sample,DivBy10K,"[NIRv, fPAR, MCTI, LAI, 1384.7793, 1369.7537, 1364.7452, 1895.65, 1394.7964, 1359.7366]"


In [50]:
from pandas import DataFrame
def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df


In [51]:
left_align(dfSummary)

,Group,MinBands,#Bands,#Trials,Randomizer,Scaling,Top10Rank
0,ALL,200,447,10897,randit,DivBy10K,"['NIRv', 'fPAR', 'LAI', 'MCTI', '1910.6757', '1900.6586', '1880.6244', '1905.6671', '1374.7622', '1885.633']"
1,ALL,100,447,5737,randit,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1910.6757', '1880.6244', '1374.7622', '1900.6586', '1895.65', '1885.633']"
2,ALL,50,447,3227,sample,DivBy10K,"['NIRv', 'LAI', 'MCTI', '1910.6757', '1880.6244', '1374.7622', '1379.7708', '1885.633', '1905.6671', '1900.6586']"
3,ALL,10,447,922,randit,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1364.7452', '1359.7366', '383.072', '1379.7708', '1374.7622', '1885.633']"
4,ALL,10,447,1088,sample,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1384.7793', '1364.7452', '1359.7366', '1369.7537', '1394.7964', '1895.65']"
5,ALL,10,447,1089,sample,DivBy10K,"['NIRv', 'fPAR', 'LAI', 'MCTI', '1369.7537', '1364.7452', '1359.7366', '1384.7793', '1394.7964', '1895.65']"
6,ALL,10,447,1088,sample,DivBy10K,"['NIRv', 'LAI', 'MCTI', '1369.7537', '1359.7366', '1384.7793', '1364.7452', '1394.7964', '1895.65', '1374.7622']"
7,ALL,10,447,1088,sample,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1369.7537', '1359.7366', '1384.7793', '1364.7452', '1394.7964', '1895.65']"
8,ALL,10,447,1088,sample,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1359.7366', '1364.7452', '1369.7537', '1384.7793', '1394.7964', '1910.6757']"
9,ALL,10,447,1088,sample,DivBy10K,"['NIRv', 'fPAR', 'MCTI', 'LAI', '1384.7793', '1369.7537', '1364.7452', '1895.65', '1394.7964', '1359.7366']"


In [52]:
dfSort = dfTest.sort_values(["Group","ID"])
dfSort

,Group,Top10Index,Top10Rank,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
1,ALL,"[15, 7, 6, 8, 326, 320, 219, 324, 323, 321]","[NIRv, LAI, fPAR, MCTI, 1910.6757, 1880.6244, 1374.7622, 1900.6586, 1895.65, 1885.633]",DivBy10K,randit,447,5737,10,100,96:13:25,ilab213,40,Pool,randint_sets_20240211_collection-ALL_max_occurrences-100_batch_size-10_num_samples-5737_bandLen-447,218557492334736085,ALL on ilab213 - min 100 occurrences: ~96 hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
4,ALL,"[15, 7, 6, 8, 221, 217, 216, 218, 223, 323]","[NIRv, LAI, fPAR, MCTI, 1384.7793, 1364.7452, 1359.7366, 1369.7537, 1394.7964, 1895.65]",DivBy10K,sample,447,1088,10,10,10:13:32.980461,gpu001,40,Pool,202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax,225919640119481430,"### gpu004: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
13,ALL,"[218, 310, 221, 216, 217, 323, 327, 223, 161, 321]","[1369.7537, 1830.5391, 1384.7793, 1359.7366, 1364.7452, 1895.65, 1915.6842, 1394.7964, 1084.2671, 1885.633]",minmax,sample,447,1088,10,10,9:34:11.607509,gpu003,40,Pool,20240113_gpu,342218863809278249,"GPU: Epochs = 3 and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized (""scale_data_method"": ""minmax"")",hyperspectral-aggregate.separate-background.tif,EarlyWarning
5,ALL,"[15, 6, 7, 8, 218, 217, 216, 221, 223, 323]","[NIRv, fPAR, LAI, MCTI, 1369.7537, 1364.7452, 1359.7366, 1384.7793, 1394.7964, 1895.65]",DivBy10K,sample,447,1089,10,10,9:44:57.880395,gpu001,40,Pool,202410125-gpu001-callbacks-divideby10k_noMinMax,393911649901623133,"### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized (""scale_data_method"": ""None"")",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
11,ALL,"[310, 218, 323, 327, 217, 216, 223, 221, 322, 219]","[1830.5391, 1369.7537, 1895.65, 1915.6842, 1364.7452, 1359.7366, 1394.7964, 1384.7793, 1890.6415, 1374.7622]",minmax,sample,447,1088,10,10,9:36:55.985009,gpu003,40,Pool,20240118-callbacks-gpu,504373152637067057,"gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized (""scale_data_method"": ""minmax"")",hyperspectral-aggregate.separate-background.tif,EarlyWarning
9,ALL,"[15, 6, 8, 7, 221, 218, 217, 323, 223, 216]","[NIRv, fPAR, MCTI, LAI, 1384.7793, 1369.7537, 1364.7452, 1895.65, 1394.7964, 1359.7366]",DivBy10K,sample,447,1088,10,10,NoMinutes,"ilab[201-212,211]",10,pdsh,202410120-callbacks-divideby10k_noMinMax,648862663793551468,"### CPU pdsh[201-212,211]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized",hyperspectral-aggregate.separate-background-scaled.tif,EarlyWarning
12,ALL,"[323, 218, 310, 327, 216, 221, 217, 223, 322, 219]","[1895.65, 1369.7537, 1830.5391, 1915.6842, 1359.7366, 1384.7793, 1364.7452, 1394.7964, 1890.6415, 1374.7622]",minmax,sample,447,1088,10,10,9:34:11.607509,gpu003,40,Pool,20240117-callbacks-gpu,660588493840123655,"gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized (""scale_data_method"": ""minmax"")",hyperspectral-aggregate.separate-background.tif,EarlyWarning
14,ALL,"[217, 218, 161, 216, 221, 166, 144, 118, 164, 155]","[1364.7452, 1369.7537, 1084.2671, 1359.7366, 1384.7793, 1109.3098, 999.122, 868.9001, 1099.2927, 1054.216]",minmax,sample,447,1088,10,10,9:34:11.607509,gpu003,40,Pool,20240112,789617681324607598,"CPU & GPU: Epochs = 2 and explainer.shap_values(X.iloc[0:50, :], nsamples=500) - All bands normalized (""scale_data_method"": ""minmax"")",hyperspectral-aggregate.separate-background.tif,EarlyWarning
3,ALL,"[15, 7, 6, 8, 217, 216, 21, 220, 21